In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
from tsforest.forecaster import LightGBMForecaster

import matplotlib.pyplot as plt
import seaborn as sns

# local modules
import sys
sys.path.append("../lib/")
from utils import compute_scaling, reduce_mem_usage
from evaluation import WRMSSEEvaluator

/home/turing/miniconda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
data = (pd.read_parquet("../input/train_dataframe.parquet")
        .dropna()
        .reset_index(drop=True)
        .rename({"q":"y"}, axis=1)
       )

***
validation periods

In [5]:
def make_valid_periods(end_date, valid_length, n_folds):
    right_date = pd.to_datetime(end_date)
    valid_periods = list()
    
    for i in range(n_folds):
        left_date = right_date - pd.DateOffset(days=valid_length-1)
        valid_periods.append((left_date, right_date))
        right_date = left_date - pd.DateOffset(days=1)
    
    return valid_periods[::-1]

In [28]:
valid_periods = [(pd.to_datetime("2015-04-25"), pd.to_datetime("2015-05-22")),
                 (pd.to_datetime("2015-05-23"), pd.to_datetime("2015-06-19")),
                 (pd.to_datetime("2016-02-29"), pd.to_datetime("2016-03-27")),
                 (pd.to_datetime("2016-03-28"), pd.to_datetime("2016-04-24"))]
valid_periods

[(Timestamp('2015-04-25 00:00:00'), Timestamp('2015-05-22 00:00:00')),
 (Timestamp('2015-05-23 00:00:00'), Timestamp('2015-06-19 00:00:00')),
 (Timestamp('2016-02-29 00:00:00'), Timestamp('2016-03-27 00:00:00')),
 (Timestamp('2016-03-28 00:00:00'), Timestamp('2016-04-24 00:00:00'))]

***
building the models

In [32]:
# 5 years of history
train_history = 1500

In [33]:
time_features = [
    "year",
    "month",
    "year_week",
    "week_day",
    "month_progress", 
]

model_kwargs = {
    "time_features":time_features,
    "lags": list(range(1,15)),
    "window_functions":["mean","std"],
    "window_sizes":[7,14,28,56,84],
    "exclude_features":["regular_price",
                        "ts_id",
                        "event_type_1",
                        "event_name_2",
                        "event_type_2"],
    "categorical_features":{#"ts_id":"default",
                            "item_id":"default", 
                            "dept_id":"default",
                            "cat_id":"default",
                            "store_id":"default",
                            "state_id":"default",
                            "event_name_1":"default", 
                            #"event_type_1":"default", 
                            #"event_name_2":"default", 
                            #"event_type_2":"default",
                            "snap":"default"},
    "ts_uid_columns":["item_id","store_id"]
}

In [34]:
%%time

models = list()

for i,valid_period in enumerate(valid_periods):
    print(f" {i+1}/{len(valid_periods)} ".center(100, "#"))
    print(f" Validation period: {valid_period} ".center(100, "#"))
    print("#"*100)
    
    valid_start = valid_period[0]
    valid_end = valid_period[1]
    train_start = valid_start - pd.DateOffset(days=train_history)
        
    _train_data = data.query("ds <= @valid_end").reset_index(drop=True)
    _valid_index = _train_data.query("@valid_start <= ds <= @valid_end").index

    _fcaster = LightGBMForecaster(**model_kwargs)
    _fcaster.prepare_features(train_data=_train_data, valid_index=_valid_index);
    _fcaster.train_features.dropna(inplace=True)
    _fcaster.train_features = _fcaster.train_features.query("ds >= @train_start")
    _fcaster.train_features = reduce_mem_usage(_fcaster.train_features)
    _fcaster.valid_features = reduce_mem_usage(_fcaster.valid_features)
    _fcaster.train_data = _fcaster.train_data.query("ds >= @train_start")
    
    with open(f"model{i}.pickle", "wb") as handler:
        pickle.dump(_fcaster, handler, protocol=4)
        handler.close()


############################################### 1/4 ################################################
##### Validation period: (Timestamp('2015-04-25 00:00:00'), Timestamp('2015-05-22 00:00:00')) ######
####################################################################################################
############################################### 2/4 ################################################
##### Validation period: (Timestamp('2015-05-23 00:00:00'), Timestamp('2015-06-19 00:00:00')) ######
####################################################################################################
############################################### 3/4 ################################################
##### Validation period: (Timestamp('2016-02-29 00:00:00'), Timestamp('2016-03-27 00:00:00')) ######
####################################################################################################
############################################### 4/4 #######################################

***

In [35]:
# precomputed (features) models
precomputed_models = list()
for i in range(4):
    with open(f"model{i}.pickle", "rb") as handler: 
        model = pickle.load(handler)
        precomputed_models.append(model)
        handler.close()   

***